## Collecte de données

In [9]:
!sudo apt update
!sudo apt install python3-pip
!pip install Pillow

Password:
sudo: a password is required
Password:
sudo: a password is required
zsh:1: command not found: pip


In [18]:
!pip3 install --user Pillow sparqlwrapper pandas requests

     |████████████████████████████████| 71kB 3.5MB/s eta 0:00:011
     |████████████████████████████████| 163kB 9.6MB/s eta 0:00:01
     |████████████████████████████████| 122kB 13.4MB/s eta 0:00:01
     |████████████████████████████████| 71kB 13.7MB/s eta 0:00:01
     |████████████████████████████████| 122kB 21.4MB/s eta 0:00:01
You should consider upgrading via the 'pip install --upgrade pip' command.


In [19]:
import os 
import json
from PIL import Image
import sys
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON
import requests
import shutil

/Users/julieludwig/Library/Python/3.8/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


On créé un dossier image et on télécharge 100 images de notre choix.

In [25]:
#os.mkdir("images")
endpoint_url = "https://query.wikidata.org/sparql"

query= """
SELECT DISTINCT ?horse ?horseLabel ?image WHERE {
    ?horse wdt:P31 wd:Q726.
     ?horse wdt:P18 ?image. 
    SERVICE wikibase:label { bd:serviceParam wikibase:language "fr". }
    }
LIMIT 100
"""
def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (
        sys.version_info[0],
        sys.version_info[1],
    )
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

array=[]
results = get_results(endpoint_url, query)

for result in results["results"]["bindings"]:
    array.append(
            (
                    result["horseLabel"]["value"],
                    result["image"]["value"],
            )
    )
dataframe = pd.DataFrame(array, columns=["car", "image"])
dataframe = dataframe.astype(
        dtype={"car": "<U200", "image": "<U200"}
)

def download_image(url, folder_path):
    headers = {"User-Agent": "mozilla/5.0"}
    request = requests.get(url, allow_redirects=True, headers=headers, stream=True)
    if request.status_code == 200:
        filename = os.path.basename(url)
        filepath = os.path.join(folder_path, filename)
        with open(filepath, "wb") as image:
            request.raw.decode_content = True
            shutil.copyfileobj(request.raw, image)
    return request.status_code

folder_path = "./images"
dataframe.image.apply(lambda url: download_image(url, folder_path))

0     200
1     200
2     200
3     200
4     200
     ... 
95    200
96    200
97    200
98    200
99    200
Name: image, Length: 100, dtype: int64

In [ ]:
#récupération des données exif d'une image
def get_image_metadata(image_path):
    with Image.open(image_path) as img:
        width, height = img.size
        format = img.format
        exif_data = img._getexif()
        orientation = exif_data.get(0x0112) if 0x0112 in exif_data else 'N/A'
        creation_date = exif_data.get(0x0132, 'N/A') if 0x0132 in exif_data else 'N/A'
        camera_model = exif_data.get(0x0110, 'N/A') if 0x0110 in exif_data else 'N/A'

        metadata = {
            'filename': os.path.basename(image_path),
            'size': os.path.getsize(image_path),
            'format': format,
            'orientation': orientation,
            'width': width,
            'height': height,
            'creation_date': creation_date,
            'camera_model': camera_model
            # Ajoutez d'autres métadonnées si nécessaire
        } 
        return metadata
    
#parcourt les images du dossier image et crée un tableau avec les infos de ces images
dossier_images = "images"
metadata_images = []

for filename in os.listdir(dossier_images):
    if filename.endswith((".jpg", ".jpeg", ".png")):
        image_path = os.path.join(dossier_images, filename)
        metadata = get_image_metadata(image_path)
        metadata_images.append(metadata)

#tableau metadata_images dans le fichier json
json_path = "metadata_images.json"
with open(json_path, 'w') as json_file:
    json.dump(metadata_images, json_file, indent=4)
print("ok")